Imports

In [1]:
#basics:
import pandas as pd
import numpy as np

#sklearn:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#tensorflow: 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


2024-06-04 16:18:58.433931: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 16:19:02.245708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Loding data

In [2]:
!pwd


/home/alish/code/amelietatin/Predicting_land_cover/notebooks


In [3]:
df= pd.read_csv("/home/alish/code/amelietatin/Predicting_land_cover/data/interpolated_final_no_water.csv")

In [4]:
df.drop("Unnamed: 0.1", axis=1, inplace=True)
df.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
df

,SITECODE,date,Bare Ground,Built Area,Crops,Flooded Vegetation,Grass,Shrub and Scrub,Snow and Ice,Trees,Water,lat,lon,temperature_quarterly_mean,precipitation_quarterly_mean,water-vapor-pressure_quarterly_mean,cloud-cover_quarterly_mean
0,AT1301000,2015-07-01,0.004731,0.306967,9.740933,0.434059,3.750812,1.101940,0.000000,79.647261,4.151655,48.172354,16.530699,18.86,2.788798e-08,942.75470,0.536892
1,AT1301000,2015-10-01,0.369930,0.000000,9.580551,0.000000,5.368118,0.000000,0.032538,75.791308,5.394485,48.176358,16.224738,18.12,3.174909e-08,948.60770,0.553719
2,AT1301000,2016-01-01,1.024919,0.599189,9.420169,0.724955,6.985424,1.552991,0.000000,71.935356,6.637315,48.313875,16.399310,18.12,3.174909e-08,948.60770,0.553719
3,AT1301000,2016-04-01,0.334118,0.199806,8.961541,0.216983,5.762732,0.793234,0.000000,78.101251,5.523506,46.591177,13.696092,13.15,5.000581e-08,891.69970,0.644521
4,AT1301000,2016-07-01,0.010046,0.170694,7.763708,0.156480,6.612075,0.171722,0.000000,79.428145,4.506132,46.603491,13.503149,13.15,5.000581e-08,891.69970,0.644521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24945,SE0820616,2023-04-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.996812,0.000000,0.000000,39.045401,-5.357648,11.88,2.961120e-08,866.72100,0.499609
24946,SE0820616,2023-07-01,0.000000,0.000000,0.073207,0.698822,0.384433,6.433196,0.000000,90.517663,1.557420,39.032296,-6.225992,12.22,2.715160e-08,850.98413,0.501889
24947,SE0820616,2023-10-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.985115,0.005196,0.000000,39.329885,-6.779056,12.91,2.512214e-08,846.98320,0.500707
24948,SE0820616,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.996812,0.000000,0.000000,38.577682,-6.033268,12.89,2.135876e-08,857.80835,0.481869


In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the features to be used
features = ['temperature_quarterly_mean', 'precipitation_quarterly_mean', 'water-vapor-pressure_quarterly_mean', 'cloud-cover_quarterly_mean']

# Prepare the sequences
site_codes = df['SITECODE'].unique()  # unique site codes
sequences = []
targets = []
for site_code in site_codes:
    site_data = df[df['SITECODE'] == site_code].sort_values('date')  # get weather data sorted by date for one site
    sequences.append(site_data[features].values)
    targets.append(site_data[['Grass']].values)

# Pad sequences to ensure uniform length
max_seq_length = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_seq_length, dtype='float32', padding='post')
targets = pad_sequences(targets, maxlen=max_seq_length, dtype='float32', padding='post')

# Convert to numpy arrays
sequences = np.array(sequences)
targets = np.array(targets)


In [8]:
sequences.shape


(703, 36, 4)

In [9]:
targets.shape

(703, 36, 1)

In [10]:
X_train = sequences[0:499,0:28,:]
X_test = sequences[499:,29:,:]
y_train = targets[0:499,0:28,:]
y_test = targets[499:,29:,:]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((499, 28, 4), (204, 7, 4), (499, 28, 1), (204, 7, 1))

In [11]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(28,4)))
model.add(LSTM(50, return_sequences=True))
model.add(Dense(1))  # Output layer with 9 neurons for the 9 targets

model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


2024-06-04 16:19:19.107754: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-04 16:19:19.108645: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/alish/.pyenv/versions/3.10.6/envs/predicting_land_cover/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - loss: 86.6035 - val_loss: 68.8684
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 79.9780 - val_loss: 57.7797
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 82.1924 - val_loss: 54.6190
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 78.3418 - val_loss: 54.3693
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 73.2924 - val_loss: 54.6261
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 65.0266 - val_loss: 54.6065
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 66.9340 - val_loss: 54.0738
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 71.6846 - val_loss: 53.8552
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 70.7019 - val_loss: 54.2333
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 76.5943 - val_loss: 54.5098
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 71.8265 - val_loss: 54.0746
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s

In [13]:
# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Optionally, you can measure additional metrics such as RMSE or R2 score
from sklearn.metrics import mean_squared_error, r2_score
 
rmse = np.sqrt(mean_squared_error(y_test.reshape(-1, 1), predictions.reshape(-1, 1)))
r2 = r2_score(y_test.reshape(-1, 1), predictions.reshape(-1, 1))

print(f'Test RMSE: {rmse}')
print(f'Test R2 Score: {r2}')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 112.7685
Test Loss: 119.5051040649414
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Test RMSE: 10.931838989257812
Test R2 Score: 0.0008435845375061035


In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(28,4)))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(Dense(9))  # Output layer with 9 neurons for the 9 targets

# Compile the model with Adam optimizer and lower learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Optionally, you can measure additional metrics such as RMSE or R2 score
from sklearn.metrics import mean_squared_error, r2_score

rmse = np.sqrt(mean_squared_error(y_test.reshape(-1, 9), predictions.reshape(-1, 9)))
r2 = r2_score(y_test.reshape(-1, 9), predictions.reshape(-1, 9))

print(f'Test RMSE: {rmse}')
print(f'Test R2 Score: {r2}')


Epoch 1/50


/home/alish/.pyenv/versions/3.10.6/envs/predicting_land_cover/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 91ms/step - loss: 536.3773 - val_loss: 485.0044
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 464.4578 - val_loss: 440.3761
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 440.7169 - val_loss: 420.4216
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 419.4058 - val_loss: 406.6977
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 410.4508 - val_loss: 395.4969
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 396.3726 - val_loss: 386.2119
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 379.5963 - val_loss: 378.1819
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 372.4604 - val_loss: 371.0970
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 366.0907 - val_loss: 365.0426
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 364.1987 - val_loss: 359.5725
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 371.7510 - val_loss: 354.7162
Epoch 12/50
13/13 ━━━━━━━━━━━━

In [87]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define the LSTM model with increased dropout rate and early stopping
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(28,4)))
model.add(Dropout(0.5))  # Increased dropout rate
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))  # Increased dropout rate
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))  # Increased dropout rate
model.add(Dense(9))

# Compile the model with RMSprop optimizer and adjusted learning rate
optimizer = RMSprop(learning_rate=0.0001)  # Adjusted learning rate
model.compile(optimizer=optimizer, loss='mse')

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Calculate additional metrics
rmse = np.sqrt(mean_squared_error(y_test.reshape(-1, 9), predictions.reshape(-1, 9)))
r2 = r2_score(y_test.reshape(-1, 9), predictions.reshape(-1, 9))

print(f'Test RMSE: {rmse}')
print(f'Test R2 Score: {r2}')


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - loss: 559.9285 - val_loss: 548.2737
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 544.2148 - val_loss: 533.5867
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 534.4765 - val_loss: 520.2392
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 528.0717 - val_loss: 509.2612
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 512.2615 - val_loss: 500.2188
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 508.8994 - val_loss: 492.0312
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 511.3166 - val_loss: 485.0446
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 483.3940 - val_loss: 478.7904
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 487.8213 - val_loss: 473.0283
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 473.5944 - val_loss: 467.8767
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 482.6838 - val_loss: 463.1828
Epoch 12/50
13/13 ━

In [107]:
# Define the LSTM model with increased dropout rate and early stopping
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(29,5)))  # Adjusted input shape for the additional feature
model.add(Dropout(0.5))  # Increased dropout rate
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))  # Increased dropout rate
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))  # Increased dropout rate
model.add(Dense(9))

# Compile the model with RMSprop optimizer and adjusted learning rate
optimizer = RMSprop(learning_rate=0.0001)  # Adjusted learning rate
model.compile(optimizer=optimizer, loss='mse')

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Calculate additional metrics
rmse = np.sqrt(mean_squared_error(y_test.reshape(-1, 9), predictions.reshape(-1, 9)))
r2 = r2_score(y_test.reshape(-1, 9), predictions.reshape(-1, 9))

print(f'Test RMSE: {rmse}')
print(f'Test R2 Score: {r2}')


Epoch 1/50


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 4 and 5 for '{{node sequential_8_1/lstm_20_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_8_1/lstm_20_1/strided_slice_2, sequential_8_1/lstm_20_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [?,4], [5,400].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(None, 4), dtype=float32)
  • states=('tf.Tensor(shape=(None, 100), dtype=float32)', 'tf.Tensor(shape=(None, 100), dtype=float32)')
  • training=True

In [ ]:
from huggingface_hub import hf_hub_download
import torch
from transformers import TimeSeriesTransformerForPrediction

file = hf_hub_download(
    repo_id="hf-internal-testing/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
)
batch = torch.load(file)

model = TimeSeriesTransformerForPrediction.from_pretrained(
    "huggingface/time-series-transformer-tourism-monthly"
)

# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
)

loss = outputs.loss
loss.backward()

# during inference, one only provides past values
# as well as possible additional features
# the model autoregressively generates future values
outputs = model.generate(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_time_features=batch["future_time_features"],
)

mean_prediction = outputs.sequences.mean(dim=1)